<h1><center>IBM Data Science Capstone Project Final Submission</center></h1>

<h4> <center>This is the final submission in the IBM Data Science Capstone Project</center> </h4>

<h2> Week 4 Submission <h2> 

<h3>Business Understanding</h3>
<h5> For this project we are doing a case study on an Italian immigrant moving to Canada. He is a entrepreneur who want to start an Italian restaurant in Canada that serves authentic home cooked style meal rather than commercial chain.
    <p>The requirements that were specified by the client are as follows, </p>
    <ol>
        <li>Client wants to open his restaurant in one of the five largest city by Population. </li>
        <li>Client wants to know the open his restaurant in or near downtown area.</li>
        <li>Client wants to know the Italian restaurants nearby to gage competition.</li>
        <li>Client wants to know how the existing Italian restaurant are performing. </li>
        <li>Client wants to know the Italian (country of origin) population as he will be serving Italian home cooked meal.</li>        
        <li>Client wants to know the estimated rental price of the restaurant place (area is approximately 1000 -1500 sq.ft).</li>
        <li>Client wants to know the crime rate in the downtown areas to know if its safe.</li>
    </ol>

<h3>Analytic Approach</h3>
<h5>From the requirements given by the client, our job is to find a city and place where our client can successfully open and run his Italian restaurant. For this we need to collect data from various sources on the population data,and people of Italian origin , what areas constitute as downtown, crime data in these area, find the trends and like for restaurants in the downtown are and lastly estimate the rental price for a space 1000 sq.ft to 1500 sq.ft
The approach chosen is as follows:
    <ol>
        <li>Collect population size of 5 largest cities in Canada and population demographics for Italian origin people. This will give us insight into possible customer flow for our client.</li>
        <li>List of neighborhood in these cities in the downtown.Use clustering method.</li>
        <li>Get the crime rate data for these areas. Use clustering method to group crimes.</li>
        <li>Existing Italian restaurant in these neighborhood and like they received. Use statistical method.</li>
        <li>What's trending in this areas.Use statistical method.</li>
        <li>Estimate rental prices based on square foot area and estimate the price for restaurant.Use statistical method use linear regression</li>        
    </ol>
</h5>

<h3>Data Requirement</h3>
<h5>The required data for this analysis 
    <ol>
        <li>Population and demographic data for various cities in Canada</li>
        <li>List of neighborhoods in cities in Canada</li>
        <li>Data on crime rates from the given cities</li>
        <li>Data on Italian restaurant and likes they received </li>
        <li>Data on trends in the downtown area of top 5 cities</li>
        <li>Data on rental price of commercial location in downtown of these cities </li>
    </ol></h5>

<body>
    <h3> Data Collection </h3>
    <h5><p>Since the client asked variety of data and there is no single source for collecting such data. Various sources are available to get data. Here is are some of the sources of data</p>
    </h5>
       <table>
            <tr><th>Data Required</th>
                <th colspan = "1">Data Source</th>
                <th colspan = "1">Link </th>
            </tr>
            <tr>            
                <td>Population and Demographics</td>
                <td> Wikipedia Page </td>
                <td> <a href = "https://en.wikipedia.org/wiki/Demographics_of_Canada">Wikipedia page</a></td>
            </tr>   
            <tr>            
                <td>Downtown Neighborhoods</td>
                <td> Wikipedia Page </td>
                <td> <a href = "https://en.wikipedia.org/wiki/Downtown_Toronto">Wikipedia page</a></td>
            </tr> 
            <tr>            
                <td>Crime Rate</td>
                <td> Open Data from Various Cities </td>            
            </tr>        
            <tr>            
                <td>Venues and likes in downtown of cities</td>
                <td> Foursquare API </td>
                <td> <a href = "https://developer.foursquare.com/docs/api/endpoints">Wikipedia page</a></td>
            </tr> 
            <tr>            
                <td>Trends in downtown of cities</td>
                <td> Foursquare API </td>
                <td> <a href = "https://developer.foursquare.com/docs/api/endpoints">Foursquare page</a></td>
            </tr> 
            <tr>            
                <td>Rental Prices from downtown to various cities</td>
                <td> Rental Websites like Space lease, kijiji </td>
                <td> <a href = "https://www.spacelist.ca/listings/ab/calgary">Spacelist page</a></td>
            </tr>         
         </table>
<body>
        

In [1]:
#import various libraries
import pandas as pd # this is important in any analysis
import requests as rq # this is imported to request the website data
import numpy as np #for any data science analysis


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [2]:
!pip install beautifulsoup4
!pip install lxml

     |████████████████████████████████| 112kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 7.3MB/s eta 0:00:01


In [3]:
# get population on various cities in canada
population_page = rq.get("https://www.worldatlas.com/articles/biggest-cities-in-canada.html")
print("status of the population page is:" ,population_page.status_code)

#get table information from this page
from bs4 import BeautifulSoup
pop_soup = BeautifulSoup(population_page.text, 'html.parser')

status of the population page is: 200


In [4]:
header = pop_soup.find('tr')
colname = [col.get_text() for col in header.find_all('th')]
df_pop = pd.DataFrame(columns=colname)
df_pop

,Rank,﻿City,Population


In [5]:
n_rows = []
Areas  = pop_soup.find_all('tr')
for Area in Areas:
    rows = [row.get_text() for row in Area.find_all('td')]
    n_rows.append(rows)
    df_pop = pd.DataFrame(n_rows)

df_pop.columns = colname
df_pop.dropna(axis=0,inplace=True)

,Rank,﻿City,Population
1,1,"Toronto, Ontario","5,429,524"
2,2,"Montreal, Quebec","3,519,595"
3,3,"Vancouver, British Columbia","2,264,823"
4,4,"Calgary, Alberta","1,237,656"
5,5,"Edmonton, Alberta","1,062,643"
6,6,"Ottawa–Gatineau, Ontario/Quebec","989,657"
7,7,"Winnipeg, Manitoba","711,925"
8,8,"Quebec City, Quebec","705,103"
9,9,"Hamilton, Ontario","693,645"
10,10,"Kitchener–Waterloo–Cambridge, Ontario","470,015"


<h5>Get the people from Italian country of origin</h5>

In [18]:
#demographics page
demo_page = rq.get("https://en.wikipedia.org/wiki/Italian_Canadians")
print("page status for Italian Canadians:", demo_page.status_code)

demo_soup = BeautifulSoup(demo_page.text,'html.parser')


page status for Italian Canadians: 200


In [19]:
demo_table = demo_soup.findAll('table')[3]

In [20]:
demo_header =  demo_table.find('tr')
colname1 = [col.get_text() for col in demo_header.find_all('th')]
for col in colname1:
    coln = col.strip("\n")

df_demo = pd.DataFrame(columns=colname1)
colname1[7] = 'Italian Population (2016)'
colname1[8] = '% of ethnic population(2016)'
colname1[0] = 'City'
df_demo

,Metropolitan area\n,Population (2001)[23]\n,% of ethnic population (2001)\n,Population (2006)[24]\n,% of ethnic population (2006)\n,Population (2011)[25]\n,% of ethnic population (2011)\n,Population (2016)[26]\n,% of ethnic population (2016)\n


In [21]:
demo_rows = []
dems  = demo_table.find_all('tr')
for dem in dems:
    rows = [row.get_text() for row in dem.find_all('td')]
    demo_rows.append(rows)
    df_demo = pd.DataFrame(demo_rows)

#df_pop.columns = colname
df_demo.dropna(axis=0,inplace=True)
df_demo = df_demo.replace('\n','',regex=True)
df_demo = df_demo.replace('CMA','',regex=True)
df_demo.columns = colname1
df_demo

,City,Population (2001)[23]\n,% of ethnic population (2001)\n,Population (2006)[24]\n,% of ethnic population (2006)\n,Population (2011)[25]\n,% of ethnic population (2011)\n,Italian Population (2016),% of ethnic population(2016)
1,Toronto [note 2],"429,380[note 3]",9.2%,"466,155[note 4]",9.2%,"475,090[note 5]",8.6%,"484,360[note 6]",8.3%
2,Montreal,"224,460",6.6%,"260,345",7.3%,"263,565",7.0%,"279,795",7.0%
3,Greater Vancouver,"69,000",3.5%,"76,345",3.6%,"82,435",3.6%,"87,875",3.6%
4,Hamilton,"67,685[note 7]",10.3%,"72,440[note 8]",10.6%,"75,900[note 9]",10.7%,"79,725[note 10]",10.8%
5,National Capital Region,"37,435",3.6%,"45,005",4.0%,"47,975",4.0%,"53,825",4.1%
6,Niagara Region,"44,645",12.0%,"48,850",12.7%,"48,530",12.6%,"49,345",12.4%
7,Greater Calgary,"29,120",3.1%,"33,645",3.1%,"36,875",3.1%,"42,940",3.1%
8,Greater Edmonton,"22,385",2.4%,"28,805",2.8%,"29,580",2.6%,"33,800",2.6%
9,Windsor,"30,680",10.1%,"33,725",10.5%,"30,880",9.8%,"33,175",10.2%
10,Oshawa,"13,990[note 11]",4.8%,"18,225[note 12]",5.6%,"20,265[note 13]",5.8%,"22,870[note 14]",6.1%


<h5>We need to select data from this above table to create 2016 population data </h5>

In [22]:
df_demo = df_demo.drop(df_demo.columns[1:7],axis=1)
df_demo

,City,Italian Population (2016),% of ethnic population(2016)
1,Toronto [note 2],"484,360[note 6]",8.3%
2,Montreal,"279,795",7.0%
3,Greater Vancouver,"87,875",3.6%
4,Hamilton,"79,725[note 10]",10.8%
5,National Capital Region,"53,825",4.1%
6,Niagara Region,"49,345",12.4%
7,Greater Calgary,"42,940",3.1%
8,Greater Edmonton,"33,800",2.6%
9,Windsor,"33,175",10.2%
10,Oshawa,"22,870[note 14]",6.1%


<h5> We need to make some changes in the dataframe, remove \n from columns, remove the 'note' from dataframe.</h5>
<h5> Also the national capital region is actually Ottowa</h5>

In [23]:
df_demo = df_demo.replace('National Capital Region','Ottowa', regex=True)
df_demo = df_demo.replace('Greater','', regex=True)

In [24]:
df_demo

,City,Italian Population (2016),% of ethnic population(2016)
1,Toronto [note 2],"484,360[note 6]",8.3%
2,Montreal,"279,795",7.0%
3,Vancouver,"87,875",3.6%
4,Hamilton,"79,725[note 10]",10.8%
5,Ottowa,"53,825",4.1%
6,Niagara Region,"49,345",12.4%
7,Calgary,"42,940",3.1%
8,Edmonton,"33,800",2.6%
9,Windsor,"33,175",10.2%
10,Oshawa,"22,870[note 14]",6.1%
